# YOLO Image Classification Training (NO LABELING NEEDED)

This notebook trains a YOLOv8 **classifier** to answer: "Is this a TREE or NOT?"

## What you need:
Just organize your photos into folders:
```
Google Drive/
  TreePhotos/
    tree/           <- Your vineyard + tangerine photos
    not_tree/       <- Photos of posts, poles, ground, sky
```

**NO bounding boxes or labeling required!**

In [ ]:
# Cell 1: Install YOLOv8
!pip install ultralytics -q

from ultralytics import YOLO
from google.colab import drive, files
import os
import shutil

print("YOLOv8 installed!")

In [ ]:
# Cell 2: Mount Google Drive
drive.mount('/content/drive')

# Check your folder structure
print("Looking for your photos...")
!ls -la /content/drive/MyDrive/

In [ ]:
# Cell 3: Configure paths
# ========================
# EDIT THIS to match your Google Drive folder structure:

PHOTOS_PATH = "/content/drive/MyDrive/TreePhotos"

# Expected structure:
#   TreePhotos/
#     tree/        <- all tree photos (vineyard + tangerine mixed is OK)
#     not_tree/    <- posts, poles, ground, sky, nothing

# Check if folders exist
tree_path = f"{PHOTOS_PATH}/tree"
not_tree_path = f"{PHOTOS_PATH}/not_tree"

if os.path.exists(tree_path):
    tree_count = len([f for f in os.listdir(tree_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Found {tree_count} tree photos in: {tree_path}")
else:
    print(f"ERROR: Folder not found: {tree_path}")
    tree_count = 0

if os.path.exists(not_tree_path):
    not_tree_count = len([f for f in os.listdir(not_tree_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Found {not_tree_count} not_tree photos in: {not_tree_path}")
else:
    print(f"ERROR: Folder not found: {not_tree_path}")
    print("You need photos of posts/poles/ground to teach what is NOT a tree!")
    not_tree_count = 0

print(f"\nTotal: {tree_count + not_tree_count} photos")

## If you don't have `not_tree` photos yet:

Take ~50+ photos of:
- Fence posts
- Wooden poles/stakes
- Ground/soil
- Empty sky
- Anything that is NOT a tree trunk

This teaches the model what to REJECT when the ultrasonic detects something.

In [ ]:
# Cell 4: Prepare dataset (split into train/val)
import random

DATASET_PATH = "/content/dataset"

# Create classification dataset structure
for split in ['train', 'val']:
    for cls in ['tree', 'not_tree']:
        os.makedirs(f"{DATASET_PATH}/{split}/{cls}", exist_ok=True)

def split_and_copy(source_folder, class_name):
    """Split 80/20 and copy to dataset"""
    if not os.path.exists(source_folder):
        return 0, 0
    
    images = [f for f in os.listdir(source_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)
    
    split_idx = int(len(images) * 0.8)
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]
    
    for img in train_imgs:
        shutil.copy(f"{source_folder}/{img}", f"{DATASET_PATH}/train/{class_name}/{img}")
    for img in val_imgs:
        shutil.copy(f"{source_folder}/{img}", f"{DATASET_PATH}/val/{class_name}/{img}")
    
    return len(train_imgs), len(val_imgs)

# Process both classes
train_tree, val_tree = split_and_copy(tree_path, 'tree')
train_not, val_not = split_and_copy(not_tree_path, 'not_tree')

print(f"\nDataset prepared:")
print(f"  Train: {train_tree} tree, {train_not} not_tree")
print(f"  Val: {val_tree} tree, {val_not} not_tree")
print(f"  Total: {train_tree + train_not + val_tree + val_not} images")

In [ ]:
# Cell 5: Train the classifier
# =============================

# Load YOLOv8 classification model (nano for Raspberry Pi speed)
model = YOLO('yolov8n-cls.pt')  # Classification model, not detection!

# Train
results = model.train(
    data=DATASET_PATH,
    epochs=50,           # Classification needs fewer epochs than detection
    imgsz=224,           # Standard classification size
    batch=32,
    patience=10,
    project='/content/runs/classify',
    name='tree_classifier',
    exist_ok=True,
    pretrained=True,
)

print("\n" + "="*50)
print("TRAINING COMPLETE!")
print("="*50)

In [ ]:
# Cell 6: View results
from IPython.display import Image, display

run_path = '/content/runs/classify/tree_classifier'

# Training curves
if os.path.exists(f'{run_path}/results.png'):
    display(Image(filename=f'{run_path}/results.png', width=800))

# Confusion matrix
if os.path.exists(f'{run_path}/confusion_matrix.png'):
    display(Image(filename=f'{run_path}/confusion_matrix.png', width=500))

In [ ]:
# Cell 7: Test the classifier
import glob

# Load best model
best_model = YOLO(f'{run_path}/weights/best.pt')

# Test on validation images
test_images = glob.glob(f'{DATASET_PATH}/val/tree/*.jpg')[:3]
test_images += glob.glob(f'{DATASET_PATH}/val/not_tree/*.jpg')[:3]

print("Testing classifier:")
print("="*50)

for img_path in test_images:
    results = best_model(img_path, verbose=False)
    
    # Get prediction
    for r in results:
        probs = r.probs
        top_class = r.names[probs.top1]
        confidence = probs.top1conf.item()
        
        actual = 'tree' if '/tree/' in img_path else 'not_tree'
        correct = '✓' if top_class == actual else '✗'
        
        print(f"{correct} Predicted: {top_class} ({confidence:.2%}) | Actual: {actual}")

In [ ]:
# Cell 8: Save model to Google Drive

output_dir = "/content/drive/MyDrive/TreeModels"
os.makedirs(output_dir, exist_ok=True)

# Copy best weights
src = f'{run_path}/weights/best.pt'
dst = f'{output_dir}/tree_classifier.pt'
shutil.copy(src, dst)

print(f"Model saved to: {dst}")
print("\n" + "="*50)
print("TO USE ON RASPBERRY PI:")
print("="*50)
print(f"1. Download: {dst}")
print("2. Copy to Pi: /home/pi/Working Code/Updated FULL Code Standalone/models/")
print("3. Update config.py:")
print("   YOLO_MODEL_PATH = 'models/tree_classifier.pt'")
print("   YOLO_MODE = 'classify'  # Add this line")
print("4. Restart: sudo systemctl restart vineyard_robot")

In [ ]:
# Cell 9: Download model directly
files.download(f'{run_path}/weights/best.pt')
print("Downloading best.pt - rename to tree_classifier.pt")

## How it works on the robot:

1. **HC-SR04** detects something on the side → robot stops
2. **Camera** captures image
3. **Classifier** answers: `tree` or `not_tree`
4. If `tree` → scan it (thermal, canopy photo)
5. If `not_tree` → skip it, continue forward

## Tips:
- **More photos = better accuracy** (aim for 100+ per class)
- **Variety matters**: different lighting, angles, distances
- **Include edge cases**: partial views, blurry shots
- For posts: metal poles, wooden stakes, T-posts, fence posts